In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
slim = tf.contrib.slim

In [3]:
from nets import resnet_v1
from nets.tinyface_utils import conv2d_trans
from nets.tinyface_common import ANCHOR_SIZES
from nets.tinyface import *
num_classes = 1

In [4]:
resnet_arg_scope = resnet_v1.resnet_arg_scope()
resnet_v1_50 = resnet_v1.resnet_v1_50

with slim.arg_scope(resnet_arg_scope):
    inputs = tf.placeholder(dtype=tf.float32, shape=[32, 224, 224, 3])
    end_points = resnet_v1_50(inputs, is_training=True)

res2 = end_points[1]['resnet_v1_50/block1']
res3 = end_points[1]['resnet_v1_50/block2']
res4 = end_points[1]['resnet_v1_50/block3']

num_anchors = ANCHOR_SIZES.shape[0]
num_scores = num_anchors * num_classes
num_coords = num_anchors * 4
num_features = num_scores + num_coords

batch_size = res3.get_shape()[0].value
H_res3 = res3.get_shape()[1].value
W_res3 = res3.get_shape()[2].value

dres2 = slim.conv2d(res2, num_features, 1, stride=2, padding='SAME', scope='down_res3')

ures4 = conv2d_trans(res4, filter_shape=[1, 1, num_features, 1024], strides=[1, 2, 2, 1],
                   output_shape=[batch_size, H_res3, W_res3, num_features],
                   padding='SAME', scope='up_res4')

res3 = slim.conv2d(res3, num_features, 1, stride=1, scope='res3')

fpyramid = tf.add_n([res3, dres2, ures4], name='fpyramid')


scores, localisations = tf.split(fpyramid,[num_scores, num_coords], axis=3)

In [5]:
tf.split(fpyramid,[num_scores, num_coords], axis=3)

[<tf.Tensor 'split_1:0' shape=(32, 14, 14, 25) dtype=float32>,
 <tf.Tensor 'split_1:1' shape=(32, 14, 14, 100) dtype=float32>]

In [6]:
tensor_shape(scores)[:-1]+[num_anchors, 4]

[32, 14, 14, 25, 4]

In [7]:
# solve for prediction
scores = tf.reshape(scores, tensor_shape(scores)[:-1]+[num_anchors, num_classes])
localisations = tf.reshape(localisations, tensor_shape(localisations)[:-1]+[num_anchors, 4])

TypeError: softmax() got an unexpected keyword argument 'dim'

In [15]:
a = tf.get_variable(name='b',shape=[32,14,14,25],dtype=tf.float32)

In [16]:
predictions = slim.softmax(a)

In [17]:
predictions

<tf.Tensor 'softmax_1/Reshape_1:0' shape=(32, 14, 14, 25) dtype=float32>